In [1]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

tf.version.VERSION

'2.4.0'

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

y_train = y_train[:1000]
y_test = y_test[:1000]
X_train = X_train[:1000].reshape(-1, 28*28) / 255.
X_test = X_test[:1000].reshape(-1, 28*28) / 255.

In [3]:
def create_mod():
    mod = Sequential([Dense(512, activation='relu', input_shape=(784,)),
                      Dropout(0.2),
                      Dense(10)])
    mod.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return mod

In [4]:
mod = create_mod()
mod.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training

In [5]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1)
mod.fit(X_train,
        y_train,
        epochs=10,
        validation_data=(X_test, y_test),
        callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 1s 20ms/step - loss: 1.5751 - sparse_categorical_accuracy: 0.5329 - val_loss: 0.6874 - val_sparse_categorical_accuracy: 0.8090

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4494 - sparse_categorical_accuracy: 0.8743 - val_loss: 0.5382 - val_sparse_categorical_accuracy: 0.8440

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2990 - sparse_categorical_accuracy: 0.9129 - val_loss: 0.4500 - val_sparse_categorical_accuracy: 0.8720

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2146 - sparse_categorical_accuracy: 0.9504 - val_loss: 0.4450 - val_sparse_categorical_accuracy: 0.8620

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1417 - sparse_cat

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

In [7]:
mod = create_mod()
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Untrained: Acc: {100 * acc:5.2f}')

32/32 - 0s - loss: 2.3906 - sparse_categorical_accuracy: 0.1330
Untrained: Acc: 13.30


In [8]:
mod.load_weights(checkpoint_path)
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Loaded weights: Acc: {100 * acc:5.2f}')

32/32 - 0s - loss: 0.4215 - sparse_categorical_accuracy: 0.8670
Loaded weights: Acc: 86.70


### Checkpoint Callback Options

In [9]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

BATCH = 32

In [10]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5 * BATCH)

mod = create_mod()
mod.save_weights(checkpoint_path.format(epoch=0))
mod.fit(X_train,
        y_train,
        epochs=50,
        batch_size=BATCH,
        callbacks=[cp_callback],
        validation_data=(X_test, y_test),
        verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index']

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [13]:
mod = create_mod()
mod.load_weights(latest)
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Restored mod: Acc: {100 * acc:5.2f}%')

32/32 - 0s - loss: 0.4880 - sparse_categorical_accuracy: 0.8790
Restored mod: Acc: 87.90%


### Manually save weights

In [14]:
mod.save_weights('./checkpoints/my_checkpoint')
mod = create_mod()
mod.load_weights('./checkpoints/my_checkpoint')
loss, acc = mod.evaluate(X_test, y_test, verbose=2)
print(f'Restored mod: Acc: {100 * acc:5.2f}%')

32/32 - 0s - loss: 0.4880 - sparse_categorical_accuracy: 0.8790
Restored mod: Acc: 87.90%


### Saved model format

In [15]:
mod = create_mod()
mod.fit(X_train, y_train, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.5708 - sparse_categorical_accuracy: 0.5227
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4561 - sparse_categorical_accuracy: 0.8714
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.3056 - sparse_categorical_accuracy: 0.9159
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1900 - sparse_categorical_accuracy: 0.9658
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1395 - sparse_categorical_accuracy: 0.9782


In [16]:
!mkdir -p saved_model

In [17]:
mod.save('saved_model/my_mod')

INFO:tensorflow:Assets written to: saved_model/my_mod/assets


In [19]:
!ls saved_model/my_mod

assets         saved_model.pb variables


In [20]:
new_mod = keras.models.load_model('saved_model/my_mod')
new_mod.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss, acc = new_mod.evaluate(X_test, y_test, verbose=2)
print(f'Acc: {acc:5.2f}')
print(new_mod.predict(X_test).shape)

32/32 - 0s - loss: 0.4391 - sparse_categorical_accuracy: 0.8650
Acc:  0.87
(1000, 10)


### HDF5 Format

In [23]:
mod = create_mod()
mod.fit(X_train, y_train, epochs=5)
mod.save('my_mod.h5')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.5744 - sparse_categorical_accuracy: 0.5242
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4877 - sparse_categorical_accuracy: 0.8513
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.3000 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2426 - sparse_categorical_accuracy: 0.9358
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1526 - sparse_categorical_accuracy: 0.9685


In [24]:
new_mod = keras.models.load_model('my_mod.h5')
new_mod.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
loss, acc = new_mod.evaluate(X_test, y_test, verbose=2)
print(f'Acc: {acc:5.2f}')

32/32 - 0s - loss: 0.4203 - sparse_categorical_accuracy: 0.8660
Acc:  0.87


Derived from <a href="https://www.tensorflow.org/tutorials/keras/save_and_load">TensorFlow Tutorials</a>

```
# MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.
```